In [1]:
import os 
import numpy as np
import tensorflow as tf
from tensorflow.keras.callbacks import TensorBoard
import time
from sklearn.model_selection import train_test_split
import cv2

In [2]:
print(os.getcwd())

c:\Users\LENOVO\Desktop\ai_recycling_machine


In [4]:
os.chdir(r"c:\Users\LENOVO\Desktop\ai_recycling_machine")

In [5]:
# Veriyi ve etiket dosyasını yükleme
image_directory = "dataset\dataset-resized"
labels_train_file = r"C:\Users\LENOVO\Desktop\ai_recycling_machine\labels\one-indexed-files-notrash_train.txt"
labels_test_file = r"C:\Users\LENOVO\Desktop\ai_recycling_machine\labels\one-indexed-files-notrash_test.txt"
labels_val_file = r"C:\Users\LENOVO\Desktop\ai_recycling_machine\labels\one-indexed-files-notrash_val.txt"

In [6]:
# Dosya yolunu vererek etiketleri yükleyen bir fonksiyon tanımlıyoruz
def load_labels(file_path):
    labels = {}
    with open(file_path, 'r') as f:
        for line in f.readlines():
            filename, label = line.strip().split()
            labels[filename] = int(label)
    return labels

# Etiketleri yükleyin
labels_train = load_labels(labels_train_file)
labels_test = load_labels(labels_test_file)
labels_val = load_labels(labels_val_file)

# Yüklenen etiketleri kontrol etmek için
print("Eğitim Etiketleri:", labels_train)
print("Test Etiketleri:", labels_test)
print("Doğrulama Etiketleri:", labels_val)

Eğitim Etiketleri: {'cardboard202.jpg': 3, 'paper472.jpg': 2, 'paper522.jpg': 2, 'glass189.jpg': 1, 'glass325.jpg': 1, 'metal63.jpg': 5, 'cardboard61.jpg': 3, 'trash50.jpg': 6, 'paper305.jpg': 2, 'paper199.jpg': 2, 'paper539.jpg': 2, 'plastic262.jpg': 4, 'plastic210.jpg': 4, 'glass489.jpg': 1, 'cardboard369.jpg': 3, 'glass80.jpg': 1, 'glass97.jpg': 1, 'plastic275.jpg': 4, 'glass272.jpg': 1, 'glass380.jpg': 1, 'paper96.jpg': 2, 'paper17.jpg': 2, 'glass301.jpg': 1, 'paper523.jpg': 2, 'glass66.jpg': 1, 'plastic344.jpg': 4, 'paper401.jpg': 2, 'trash88.jpg': 6, 'paper414.jpg': 2, 'metal25.jpg': 5, 'glass241.jpg': 1, 'metal231.jpg': 5, 'metal329.jpg': 5, 'plastic140.jpg': 4, 'paper252.jpg': 2, 'paper205.jpg': 2, 'paper107.jpg': 2, 'trash80.jpg': 6, 'plastic271.jpg': 4, 'cardboard62.jpg': 3, 'paper410.jpg': 2, 'metal95.jpg': 5, 'paper256.jpg': 2, 'glass282.jpg': 1, 'plastic28.jpg': 4, 'paper338.jpg': 2, 'metal404.jpg': 5, 'paper33.jpg': 2, 'paper131.jpg': 2, 'metal227.jpg': 5, 'glass413.jpg':

In [7]:
# Veriyi hazırlama
def load_images(labels):
    images = []
    labels_list = []
    for filename, label in labels.items():
        # Etiketlere göre alt klasör adını belirleme
        if label == 1:
            folder = "glass"
        elif label == 2:
            folder = "paper"
        elif label == 3:
            folder = "cardboard"
        elif label == 4:
            folder = "plastic"
        elif label == 5:
            folder = "metal"
        elif label == 6:
            folder = "trash"
        
        # Tam dosya yolunu oluşturma
        img_path = os.path.join(image_directory, folder, filename)
        
        img = cv2.imread(img_path)
        if img is None:
            print(f"Resim yüklenemedi: {img_path}")
            continue  # Bu görüntüyü atlayarak sonraki görüntüye geç
        
        img = cv2.resize(img, (224, 224))  # YOLO modelleri için uygun boyut
        images.append(img)
        labels_list.append(label)
        
    return np.array(images, dtype='float32') / 255.0, np.array(labels_list)



In [8]:
# Eğitim, test ve doğrulama verilerini yükleme
X_train, y_train = load_images(labels_train)
X_test, y_test = load_images(labels_test)
X_val, y_val = load_images(labels_val)


In [9]:
from tensorflow.keras.applications import MobileNetV2

# Önceden eğitilmiş MobileNetV2 modelini yükleyin ve yalnızca son katmanları eğitin
base_model = MobileNetV2(input_shape=(224, 224, 3), include_top=False, weights='imagenet')
base_model.trainable = False  # Tüm katmanları donduruyoruz

model = tf.keras.models.Sequential([
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(6, activation='softmax')
])

In [10]:
# Etiketleri sıfır bazlı yapmak
y_train -= 1
y_test -= 1
y_val -= 1

In [11]:
log_dir = "logs/fit/" + time.strftime("%Y%m%d-%H%M%S")

# TensorBoard callback
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

In [12]:
# Modeli derleme
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

from tensorflow.keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Modeli eğitme, TensorBoard ve EarlyStopping callback'lerini ekleyin
model.fit(X_train, y_train, epochs=30, validation_data=(X_val, y_val), callbacks=[tensorboard_callback, early_stopping])


# Modeli değerlendirme
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test doğruluğu: {accuracy * 100:.2f}%')

Epoch 1/30
56/56 [==============================] - 83s 1s/step - loss: 1.1009 - accuracy: 0.5831 - val_loss: 0.6945 - val_accuracy: 0.7409
Epoch 2/30
56/56 [==============================] - 67s 1s/step - loss: 0.7130 - accuracy: 0.7364 - val_loss: 0.6488 - val_accuracy: 0.7530
Epoch 3/30
56/56 [==============================] - 57s 1s/step - loss: 0.6054 - accuracy: 0.7805 - val_loss: 0.5168 - val_accuracy: 0.8171
Epoch 4/30
56/56 [==============================] - 55s 989ms/step - loss: 0.4826 - accuracy: 0.8320 - val_loss: 0.5005 - val_accuracy: 0.8293
Epoch 5/30
56/56 [==============================] - 57s 1s/step - loss: 0.4184 - accuracy: 0.8467 - val_loss: 0.4883 - val_accuracy: 0.8049
Epoch 6/30
56/56 [==============================] - 59s 1s/step - loss: 0.3606 - accuracy: 0.8699 - val_loss: 0.4838 - val_accuracy: 0.8262
Epoch 7/30
56/56 [==============================] - 61s 1s/step - loss: 0.3166 - accuracy: 0.8903 - val_loss: 0.4399 - val_accuracy: 0.8384
Epoch 8/30
56/56 

In [ ]:
# Modeli kaydetme
model.save('model.keras')